In [ ]:
# uncomment if running from Docker
# from beakerx import *

# uncomment if running from Conda
from beakerx.object import beakerx
import pandas as pd

vc_csv = pd.read_csv('../data/variant_counts_per_product_per_protein_effect.csv', names=['product', 'protein_effect', 'count'], header = 0)
uniqueLabels = vc_csv['protein_effect'].unique()
beakerx.uniqueLabels = uniqueLabels
print ("unique labels :\n ",uniqueLabels,"\n")
boostList = []
coreList = []
labelList = []
coreMap = {}
boostMap = {}
    

for key in uniqueLabels:
    coreMap.update({key: 0})
    boostMap.update({key:0})



for index, row in vc_csv.iterrows():
    labelList.append(row['protein_effect'])
    if row['product'] == 'BOOST':
        boostList.append(row['count'])
        boostMap.update({row['protein_effect']:(boostMap[row['protein_effect']]+ row['count']) })
        
    elif row['product'] == 'CORE':
        coreList.append(row['count'])
        coreMap.update({row['protein_effect']:(coreMap[row['protein_effect']]+ row['count']) })
        
uniqueList = set(labelList)

print ("core map :\n ",coreMap,"\n")
print ("boost map :\n ",boostMap,"\n")


beakerx.variants = {'labels': uniqueLabels, 'boost': boostList, 'core': coreList}

# build data structure for stacked bar chart
beakerx.coreMap = coreMap
beakerx.boostMap = boostMap 


In [ ]:
%%html
<div id="plotlyHorizontalBar"></div>

In [ ]:
%%javascript
require.config({paths:{'plotly':'//cdn.plot.ly/plotly-latest.min'}});
require(['plotly'], function(Plotly) {
    window.Plotly = Plotly;
});


var trace1 = {
  x: beakerx.variants.labels,
  y: beakerx.variants.boost,
  name: 'CKB BOOST',
  type: 'bar'
};

var trace2 = {
  x:  beakerx.variants.labels,
  y: beakerx.variants.core,
  name: 'CKB CORE',
  type: 'bar'
};

var data = [trace1, trace2];

var layout = {barmode: 'stack'};

Plotly.newPlot('plotlyHorizontalBar', data, layout);


# Update for stacked bar charts

In [ ]:
dataList = []
for PE in beakerx.uniqueLabels:
    trace = {"x": ['CORE','BOOST'],"y": [beakerx.coreMap[PE],beakerx.boostMap[PE]], "name" : PE, "type" :  "bar" }
    dataList.append(trace)
    
#print (dataList)    
beakerx.traces = dataList    

In [ ]:
%%javascript
beakerx.displayHTML(this, '<div id="plotlyDiv"></div>');

var data = beakerx.traces

var layout = {barmode: 'stack'};

Plotly.newPlot('plotlyDiv', data, layout);